In [1]:
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("abacus")

SLG: Checking config correctness...
SLG: Config correct!
SLG Warning: models/abacus/corpus/train.txt does not exist.
Corpus will not be loaded from file.

SLG Warning: models/abacus/vocabulary/merges.txt does not exist.
Vocabulary will not be loaded from file.



In [3]:
semiotic.corpus.build(
    split_sent=False,
    keep_source=True
)


SLG: Dataset loaded from the following files: ["Toti Rigatelli_L_1984_Gori_Praticha d'alcibra_S.txt", "Franci_R_2015_Un trattato d'abaco pisano della fine del XIII secolo (MS)_Working copy.txt", 'Simi_A_1992_Anonimo fiorentino_Regole di geometria e della cosa_S.txt', 'Arrighi_G_1967{c}_Trascelta di Giovanni di Bartolo_S.txt', 'Arrighi_G_1999_Anonimo_Algorismus_S.txt', 'Arrighi_G_1964_(ps)-Paolo_S.txt', "Arrighi_G_1989{a}_Livero de l'abbecho_S.txt", "Arrighi_G_1973_Scuola lucchese--LIbro d'abaco_S.txt", "Franci_R_1983{a}_Gilio_Questioni d'algebra_S.txt", "Jens Hoyrup - Jacopo da Firenze's Tractatus Algorismi and Early Italian Abbacus Culture (Science Networks. Historical Studies) (2007).txt", 'Arrighi_G_1967{a}_Mazzingi-fioretti_S.txt', 'Simi_A_1994_Alcibra amuchabile_S.txt', "Arrighi_G_1974_P-M-Calandri[Benedetto]_Tractato-d'abbacho_S.txt", "Franci_R_2015_Un trattato d'abaco pisano della fine del XIII secolo_S_OK.txt", 'Arrighi_G_1987{f}_Gherardi+Liber habaci_S.txt', 'Franci_R_2001_Da

In [4]:
semiotic.corpus.dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source'],
        num_rows: 21780
    })
    dev: Dataset({
        features: ['text', 'source'],
        num_rows: 1210
    })
    test: Dataset({
        features: ['text', 'source'],
        num_rows: 1211
    })
})

In [5]:
meno_corpus = semiotic.corpus.train.filter(lambda paragraph: "meno" in paragraph["text"])

100%|██████████| 22/22 [00:00<00:00, 95.86ba/s]


In [6]:
meno_corpus[:10]

{'text': ['cubi e poi dimezare i ciensi e multiplicare per se medesmo e porre sopra le cose e quello che fa la sua radice meno il dimezamento di ciensi, cotanto varrà la chosa.',
  'Ancora truovami quella di 20. Poni sia 4; dì: 4 via 4 fa 16, fino in 20 è meno 4. Dunque parti la radice per quello che è meno, cioè per 4, e viene 1 e riman 0. Però fa secondo la regola di prima spetie e troverai che 4.1/2 si è la più vicina radice di 20.',
  'la quinta parte del primo el primo ten cho’ el 1/18 e ’l sechondo 1/3 del 1/3, adonqua la quinta parte d’esso primo tiene 10/518 del sechondo gioè 1/90 e la quinta parte del tergo gioè 1/15 donqua el secondo tiene la quantitade del tergo meno 1/90 de la seconda e 1/15 del tergo. Adonqua el sechondo tiene 14/15 del tergo meno 1/90 em se medesimo em pergiò 14/15 del tergo vaso tiene la quantitade del sechondo e più 1/90 d’esso sechondo gioè 91/90. L’altro tergo vaso tiene la quantitade del sechondo e la quinta parte del primo, el quale 1/5 ène sechondo

In [18]:
semiotic.vocab.build(
    save = False,
    vocab_size=2000,
    parallel = True,
    save_step=20
    )

Computing in parallel
Normalize and jobs data...
... computed in 2.118035078048706 secs.

Build alphabet...
... computed in 0.00043272972106933594 secs.

Alphabet Size: 56
Special Tokens Size: 5
Terms to compute: 1939

Enter loop


  0%|          | 0/1939 [00:00<?, ?it/s]

Compute freq...
... computed in 0.05408000946044922 secs.

Vocabulary built

Syntagmatic and Paradigmatic updated with the new vocabulary



In [17]:
sent = "I have made my plans and I must stick to them"
sent_seq = semiotic(sent)
sent_seq.chain.labels

['i',
 'have',
 'm',
 'ad',
 'em',
 'y',
 'pl',
 'an',
 'sand',
 'im',
 'ust',
 'st',
 'ic',
 'k',
 'tothe',
 'm']

In [19]:
sent_seq = semiotic(sent)
sent_seq.chain.labels

['ihave', 'made', 'my', 'plan', 'sand', 'i', 'must', 'st', 'ick', 'to', 'them']